##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [25]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [26]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

###Préparation des matrices de traits

In [27]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [28]:
samplePrefix="MGC-150812-3000-1508140124-paradigmes"
sampleFile=samplePrefix+".csv"
goldFile="MGC-150815-total-1508150555-paradigmes.csv"
analysisPrefix="MGC-150805-total"
logfile_name=analysisPrefix+samplePrefix+".log"
logfile = codecs.open(logfile_name,mode='w',encoding="utf8")

###Préparation des cases du paradigme

In [29]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires

#Préparation du calcul des analogies

###Calcul de la différence entre deux formes

In [30]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

###Accumulation des paires appartenant à un patron

In [31]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

###Transformation d'un patron en RegExp

In [32]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

###Substitution de sortie 
???

In [33]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

In [34]:
class formesPatron:
    '''
    Accumulateur de formes correspondant à un patron pour calcul de la Généralisation Minimale (cf. MGL)
    '''
    def __init__(self):
        self.formes=[]

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterForme(self,forme):
        self.formes.append(forme)
        
    def calculerGM(self):
        minLongueur=len(min(self.formes, key=len))
        maxLongueur=len(max(self.formes, key=len))
        if debug: print (minLongueur, maxLongueur, file=logfile)
        positions=[]
        if maxLongueur>minLongueur:
            positions.append("*")
        for i in xrange(minLongueur, 0, -1):
            phonemes=set([x[-i] for x in self.formes])
            if debug: print (phonemes, file=logfile)
            if "." in phonemes:
                positions.append(".")
            else:
                positions.append("".join(fs.lattice[phonemes].extent))
        return patron2regexp(positions)

class pairePatrons:
    '''
    Accumulateur de triplets (f1,f2,patron) correspondant à une paire pour calcul des Généralisations Minimales (cf. MGL)
    '''
    def __init__(self,case1,case2):
        self.patrons1={}
        self.patrons2={}
        self.case1=case1
        self.case2=case2

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterFormes(self,forme1,forme2,patron):
#        print (forme1,forme2,patron, file=logfile)
        patron12=patron
        (pat1,pat2)=patron.split("-")
        patron21=pat2+"-"+pat1
#        print (patron12,patron21, file=logfile)
        if not patron12 in self.patrons1:
            self.patrons1[patron12]=formesPatron()
        self.patrons1[patron12].ajouterForme(forme1)
        if not patron21 in self.patrons2:
            self.patrons2[patron21]=formesPatron()
        self.patrons2[patron21].ajouterForme(forme2)
        
        
    def calculerGM(self):
        resultat1={}
        for patron in self.patrons1:
            if debug: print ("patron1", patron, file=logfile)
            resultat1[patron]=self.patrons1[patron].calculerGM()
        resultat2={}
        for patron in self.patrons2:
            if debug: print ("patron2", patron, file=logfile)
            resultat2[patron]=self.patrons2[patron].calculerGM()
        return (resultat1,resultat2) 

#Classe pour la gestion des patrons, des classes et des transformations

In [35]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n,file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print (forme, file=logfile) 
            print ("pas de patron", file=logfile)
        return sortieForme
        

##Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [36]:
def OLDrapports(paradigme):
    (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    if len(paradigme)>0:
#        for index, row in paradigme.iterrows():
#            patrons.ajouterFormes(row[0],row[1],diff(row[0],row[1]))
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
    return patrons.calculerGM()

In [37]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

###Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [38]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


In [39]:
paradigmes=pd.read_csv(sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]

In [40]:
GOLD=pd.read_csv(goldFile,sep=";",encoding="utf8")
del GOLD[u"Unnamed: 0"]
paradigmesGOLD=GOLD[paradigmes.columns.values.tolist()]

- listeCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [41]:
listeCases=paradigmes.columns.values.tolist()
listeCases.remove(u"lexeme")

In [42]:
paradigmes.stack().value_counts(dropna=True).sum()

4000

In [43]:
with open(analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

In [44]:
class paradigmeDistribution:
    '''
    Gestion des distributions dans les cases du paradigme
    '''

    def __init__(self,lexeme):
        self.lexeme=lexeme
        self.formes={i:{} for i in casesTotales}

    def ajouterFormes(self,case,formes,coef=1.0):
        for forme in formes:
            if not forme in self.formes[case]:
                self.formes[case][forme]=0
            self.formes[case][forme]+=formes[forme]*coef
            
    def normaliserDistributions(self,caseListe=casesTotales):
        normalesDistributions={i:{} for i in caseListe}
        for case in caseListe:
            total=0
            for element in self.formes[case]:
                total+=self.formes[case][element]
            for element in self.formes[case]:
                normalesDistributions[case][element]=float(self.formes[case][element])/total
        return normalesDistributions
        

In [342]:
def generateForms(lexeme):
    candidats=paradigmeDistribution(lexeme)
    casesSamples=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
    casesSamples.remove("lexeme")
    for caseDepart in casesSamples:
        formeDepart=paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]
        if debug: print (caseDepart,formeDepart, file=logfile)
#        if formeDepart!="nan":
        for case in casesTotales:
            if debug: print (case, file=logfile)
            if not isinstance(resultatsLecture[(caseDepart, case)],str):
                if "," in formeDepart:
                    formesDepart=formeDepart.split(",")
                    coef=1.0/len(formesDepart)
                    for element in formesDepart:
                        candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element),coef)
                else:
                    candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart))
            else: 
                if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [336]:
paradigmes[paradigmes["lexeme"]=="dormir"].columns[paradigmes[paradigmes["lexeme"]=="dormir"].notnull().iloc[0]].tolist()

[u'lexeme',
 u'ai3S',
 u'ii3P',
 u'inf',
 u'pI2P',
 u'pi1S',
 u'pi2S',
 u'pi3S',
 u'ps3P']

def generateForms(lexeme):
    candidats=paradigmeDistribution(lexeme)
    for caseDepart in listeCases:
        if not pd.isnull(paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]):
            formeDepart=paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]
            if debug: print (caseDepart,formeDepart, file=logfile)
            if formeDepart!="nan":
                for case in casesTotales:
                    if debug: print (case, file=logfile)
                    if not isinstance(resultatsLecture[(caseDepart, case)],str):
                        if "," in formeDepart:
                            formesDepart=formeDepart.split(",")
                            coef=1.0/len(formesDepart)
                            for element in formesDepart:
                                candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element),coef)
                        else:
                            candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart))
                    else: 
                        if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [344]:
def ajouterPoint(lexeme,forme,case):
    pointName="%s-%s-%s"%(lexeme,forme,case)
#    if not pointName in digraphe.nodes():
    tam=case[:2]
    if tam=="in": tam="inf"
    digraphe.add_node(pointName, tam=tam)
    graphe.add_node(pointName, tam=tam)
    return pointName

def ajouterFleche(pointDepart,pointSortie,coef):
    digraphe.add_edge(pointDepart,pointSortie,weight=float(coef))
    if digraphe.has_edge(pointSortie,pointDepart):
        coefGraphe=float(digraphe.edge[pointSortie][pointDepart]["weight"]+coef)/2
        graphe.add_edge(pointDepart,pointSortie,weight=coefGraphe)

In [345]:
def generateParadigms(generation1):
    lexeme=generation1.lexeme
    distributionInitiale=generation1.normaliserDistributions()
    candidats=paradigmeDistribution(lexeme)
    for caseDepart in casesTotales:
        for formeDepart in distributionInitiale[caseDepart]:
            if formeDepart:
                pointDepart=ajouterPoint(lexeme,formeDepart,caseDepart)
                coefDepart=distributionInitiale[caseDepart][formeDepart]
                if debug: print (caseDepart,formeDepart, file=logfile)
                for caseSortie in casesTotales:
                    distributionSortie=resultatsLecture[(caseDepart, caseSortie)].sortirForme(formeDepart)
                    if distributionSortie:
                        if debug: print (caseSortie,distributionSortie,distributionInitiale[caseDepart], file=logfile)
                        candidats.ajouterFormes(caseSortie,distributionSortie,distributionInitiale[caseDepart][formeDepart])
                        for formeSortie in distributionSortie:
                            pointSortie=ajouterPoint(lexeme,formeSortie,caseSortie)
                            coefSortie=distributionSortie[formeSortie]
                            ajouterFleche(pointDepart,pointSortie,float(coefDepart*coefSortie))
    return candidats

In [346]:
def generate(lexeme):
    print ("génération 1",end=", ")
    generation1=generateForms(lexeme)
    print ("génération 2",end=", ")
    generation2=generateParadigms(generation1)
    print ("génération 3")
    return generation2

In [347]:
len(paradigmes.dropna(thresh=3)["lexeme"])

501

In [ ]:
%%time
debug=False
listeTest=[u"manger",u"boire",u"dormir",u"aller",u"avoir"]
#listeTest=paradigmes.dropna(thresh=3)["lexeme"].values.tolist()
digraphe=nx.DiGraph()
graphe=nx.Graph()
for i,element in enumerate(listeTest):
    print (i,end=", ")
    generate(element)

0, génération 1, génération 2, génération 3
1, génération 1, génération 2, génération 3
2, génération 1, génération 2

In [ ]:
versionStamp=dateheure()
nx.write_dot(digraphe,u"digraphe-%s.dot"%versionStamp)
nx.write_dot(graphe,u"graphe-%s.dot"%versionStamp)

In [52]:
cliques=list(nx.algorithms.clique.find_cliques(graphe))

In [53]:
for element in listeTest:
    print (element,paradigmes[paradigmes["lexeme"]==element].stack().value_counts(dropna=True).sum()-1)
for element in listeTest:
    print (paradigmes[paradigmes["lexeme"]==element].stack())

manger 6
boire 4
dormir 8
aller 23
avoir 25
549  lexeme    manger
     fi1S      mâZ6rE
     inf         mâZe
     pI2P        mâZe
     pI2S         mâZ
     pi3P         mâZ
     ppMS        mâZe
dtype: object
108  lexeme    boire
     inf        bwar
     pc2S      bwarE
     pi1P       byvô
     ppMS         by
dtype: object
254  lexeme    dormir
     ai3S       dOrmi
     ii3P       dOrmE
     inf       dOrmir
     pI2P       dOrme
     pi1S         dOr
     pi2S         dOr
     pi3S         dOr
     ps3P        dOrm
dtype: object
39  lexeme    aller
    ai3S        ala
    fi2P        ire
    fi3S        ira
    ii1S        alE
    ii2P       alje
    ii2S        alE
    ii3S        alE
    inf         ale
    pI1P        alô
    pI2P        ale
    pI2S         va
    pc1P       irjô
    pc3S        irE
    pi1P        alô
    pi1S         vE
    pi2P        ale
    pi2S         va
    pi3P         vô
    pi3S         va
    ppFS        ale
    ppMS        ale
    ps1S         

##Comparer la sortie des cliques avec le tableau de conjugaison normal

In [165]:
seuilClique=40
#paradigmesCLIQUES=pd.DataFrame(columns=paradigmes.columns.values.tolist())
paradigmesListe=[]
for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
    if len(clique)>seuilClique:
        paradigmeClique={}
        sampleOK=True
#        print (n, len(clique))
        lexeme=clique[0].split("-")[0]
        paradigmeClique["lexeme"]=lexeme
        for element in clique:
            (lexeme,forme,taminfo)=element.split("-")
            paradigmeClique[taminfo]=forme
            if taminfo in paradigmes.columns:
                if not paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].isnull().item():
                    if paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item()!=forme:
                        sampleOK=False
                        print (paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item(),forme,end=", ")
    if sampleOK:
        paradigmesListe.append(paradigmeClique)
    else:
        print ()
paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=GOLD.columns.values.tolist())

dOrm dOr, dOrmE dOrE, dOrmi dOra, dOrme dOre, dOrmir dOre, 
Orô av6rô, OrE av6rE, Eje ave, Orô av6rô, y ava, y ave, y ave, E av, Ore av6re, avwar ave, a av, OrE av6rE, Ora av6ra, E av, ô av, a av, Ora av6ra, Eje avje, 
dOr dOrm, dOrmi dOrma, dOr dOrm, dOrmir dOrme, dOr dOrm, 
ira aj6ra, va aj, ale aje, alô ajô, alE ajE, alE ajE, ale aje, va aj, alô ajô, irjô aj6rjô, ire aj6re, ala aja, ale aje, irE aj6rE, ale aje, ale aje, vô aj, va aj, alE ajE, vE aj, alje aje, 
OrE airE, ave aise, y ai, avwar air, Eje aisje, avE aisE, avjô aisjô, Orô airô, Orô airô, Eje aise, avô aisô, Ora aira, OrE airE, Ora aira, avE aisE, ô ais, avE aisE, avE aisE, y ai, Ore aire, y ai, a E, a E, 
ira al6ra, vE al, va al, vô al, aj al, va al, irE al6rE, aj al, irjô al6rjô, va al, ire al6re, 
va vE, va vE, va vE, ale vEtir, ire vEtire, ale vEte, irE vEtirE, alE vEtE, alE vEtE, ala vEti, ira vEtira, ale vEty, ale vEty, aj vEt, vô vEt, aj vEt, ale vEte, alô vEtô, alE vEtE, alje vEtje, irjô vEtirjô, alô vEtô, 
dOrmE d

In [285]:
paradigmesCLIQUES.dropna(thresh=45).iloc[5]

lexeme      dormir
ai1P        dOrmim
ai1S         dOrmi
ai2P        dOrmit
ai2S         dOrmi
ai3P        dOrmir
ai3S         dOrmi
fi1P       dOrmirô
fi1S       dOrmirE
fi2P       dOrmire
fi2S       dOrmira
fi3P       dOrmirô
fi3S       dOrmira
ii1P        dOrmjô
ii1S         dOrmE
ii2P        dOrmje
ii2S         dOrmE
ii3P         dOrmE
ii3S         dOrmE
inf         dOrmir
is1P      dOrmisjô
is1S        dOrmis
is2P      dOrmisje
is2S        dOrmis
is3P        dOrmis
is3S         dOrmi
pI1P         dOrmô
pI2P         dOrme
pI2S           NaN
pP           dOrmâ
pc1P      dOrmirjô
pc1S       dOrmirE
pc2P      dOrmirje
pc2S       dOrmirE
pc3P       dOrmirE
pc3S       dOrmirE
pi1P         dOrmô
pi1S           NaN
pi2P         dOrme
pi2S           NaN
pi3P          dOrm
pi3S           NaN
ppFP         dOrmi
ppFS         dOrmi
ppMP         dOrmi
ppMS         dOrmi
ps1P        dOrmjô
ps1S          dOrm
ps2P        dOrmje
ps2S          dOrm
ps3P          dOrm
ps3S          dOrm
Name: 5, dty

In [167]:
GOLD.loc[GOLD["lexeme"]=="dormir"].stack()

1545  lexeme      dormir
      ai1P        dOrmim
      ai1S         dOrmi
      ai2P        dOrmit
      ai2S         dOrmi
      ai3P        dOrmir
      ai3S         dOrmi
      fi1P       dOrmirô
      fi1S       dOrmirE
      fi2P       dOrmire
      fi2S       dOrmira
      fi3P       dOrmirô
      fi3S       dOrmira
      ii1P        dOrmjô
      ii1S         dOrmE
      ii2P        dOrmje
      ii2S         dOrmE
      ii3P         dOrmE
      ii3S         dOrmE
      inf         dOrmir
      is1P      dOrmisjô
      is1S        dOrmis
      is2P      dOrmisje
      is2S        dOrmis
      is3P        dOrmis
      is3S         dOrmi
      pI1P         dOrmô
      pI2P         dOrme
      pI2S           dOr
      pP           dOrmâ
      pc1P      dOrmirjô
      pc1S       dOrmirE
      pc2P      dOrmirje
      pc2S       dOrmirE
      pc3P       dOrmirE
      pc3S       dOrmirE
      pi1P         dOrmô
      pi1S           dOr
      pi2P         dOrme
      pi2S           dOr


In [280]:
def compareGOLD(row):
    if row["lexeme"]=="dodo": 
        debug=True
    else:
        debug=False
    print (row["lexeme"],end=" : ")
    ligne=[]
    lexical=True
    for case in casesTotales:
        if debug:    
            print ("dormir",case)
            print (GOLD[GOLD["lexeme"]==row["lexeme"]][case].item())
#            print (case in row[row.isnull()])
            print ((GOLD[GOLD["lexeme"]==row["lexeme"]][case].isnull().iloc[0]))
        if not case in row:
            if debug: print ("not case")
            ligne.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==row["lexeme"]][case].item()))
            lexical=False
            if debug: print ("absence",case)
        elif case in row[row.isnull()] and not GOLD[GOLD["lexeme"]==row["lexeme"]][case].isnull().iloc[0]:
            if debug: print ("not case")
            ligne.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==row["lexeme"]][case].item()))
            lexical=False
            if debug: print ("CLIQUES +NaN",case)
        elif not case in row[row.isnull()] and GOLD[GOLD["lexeme"]==row["lexeme"]][case].isnull().iloc[0]:
            if debug: print ("not case")
            ligne.append(u"%s : %s ≠ Ø" % (case,row[case]))
            lexical=False
            if debug: print ("GOLD -NaN",case)            
        elif (row[case]==GOLD[GOLD["lexeme"]==row["lexeme"]][case].item()):
            if debug: print ("egal")
#            ligne.append(u"%s : %s" % (case,row[case]))
            if debug: print ("valeurs id")
        elif (case in row[row.isnull()]) and (GOLD[GOLD["lexeme"]==row["lexeme"]][case].isnull().iloc[0]):
            if debug: print ("NaNs")
#            ligne.append(u"%s : %s" % (case,u"ØØØ"))
            if debug: print ("deux NaN")
        else:
            if debug: print ("else")
            ligne.append(u"%s : %s ≠ %s" % (case, row[case],GOLD[GOLD["lexeme"]==row["lexeme"]][case].item()))
            lexical=False
            if debug: print (u"différence",case)
    if lexical:
        print ("OK")
    else:
        print ("DIFF")
        print (", ".join(ligne))
    print ()
    return lexical

In [282]:
%%time
debug=False
paradigmesCLIQUES.apply(compareGOLD,axis=1)

boire : OK

manger : OK

avoir : OK

aller : OK

dormir : OK

dormir : DIFF
pi1S : Ø ≠ dOr, pi2S : Ø ≠ dOr, pi3S : Ø ≠ dOr, pI2S : Ø ≠ dOr, ppMP : dOrmi ≠ Ø, ppFS : dOrmi ≠ Ø, ppFP : dOrmi ≠ Ø

CPU times: user 1.08 s, sys: 17.4 ms, total: 1.1 s
Wall time: 1.14 s


In [303]:
paradigmes[paradigmes["lexeme"]=="dormir"].stack()

254  lexeme    dormir
     ai3S       dOrmi
     ii3P       dOrmE
     inf       dOrmir
     pI2P       dOrme
     pi1S         dOr
     pi2S         dOr
     pi3S         dOr
     ps3P        dOrm
dtype: object

In [229]:
GOLD[GOLD["lexeme"]=="dormir"]

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
1545,dormir,dOrmim,dOrmi,dOrmit,dOrmi,dOrmir,dOrmi,dOrmirô,dOrmirE,dOrmire,...,NaN,NaN,NaN,dOrmi,dOrmjô,dOrm,dOrmje,dOrm,dOrm,dOrm


In [230]:
paradigmesCLIQUES[paradigmesCLIQUES["lexeme"]=="dormir"]

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
4,dormir,dOrmim,dOrmi,dOrmit,dOrmi,dOrmir,dOrmi,dOrmirô,dOrmirE,dOrmire,...,NaN,NaN,NaN,dOrmi,dOrmjô,dOrm,dOrmje,dOrm,dOrm,dOrm
5,dormir,dOrmim,dOrmi,dOrmit,dOrmi,dOrmir,dOrmi,dOrmirô,dOrmirE,dOrmire,...,dOrmi,dOrmi,dOrmi,dOrmi,dOrmjô,dOrm,dOrmje,dOrm,dOrm,dOrm


In [225]:
paradigmesGOLD

,lexeme,ai1S,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,fi2S,fi3P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,abEsE,abEsa,abEsEr,abEsa,abEs6rô,abEs6rE,abEs6re,abEs6ra,abEs6rô,...,abEse,abEse,abEse,abEse,abEsjô,abEs,abEsje,abEs,abEs,abEs
1,abandonner,abâdOnE,abâdOna,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6re,abâdOn6ra,abâdOn6rô,...,abâdOne,abâdOne,abâdOne,abâdOne,abâdOnjô,abâdOn,abâdOnje,abâdOn,abâdOn,abâdOn
2,abasourdir,abazurdi,abazurdi,abazurdir,abazurdi,abazurdirô,abazurdirE,abazurdire,abazurdira,abazurdirô,...,abazurdi,abazurdi,abazurdi,abazurdi,abazurdisjô,abazurdis,abazurdisje,abazurdis,abazurdis,abazurdis
3,abattre,abati,abati,abatir,abati,abatrô,abatrE,abatre,abatra,abatrô,...,abaty,abaty,abaty,abaty,abatjô,abat,abatje,abat,abat,abat
4,abcéder,absEdE,absEda,absEdEr,absEda,absEd6rô,absEd6rE,absEd6re,absEd6ra,absEd6rô,...,NaN,NaN,NaN,absEde,absEdjô,absEd,absEdje,absEd,absEd,absEd
5,abdiquer,abdikE,abdika,abdikEr,abdika,abdik6rô,abdik6rE,abdik6re,abdik6ra,abdik6rô,...,abdike,abdike,abdike,abdike,abdikjô,abdik,abdikje,abdik,abdik,abdik
6,abecquer,abEkE,abEka,abEkEr,abEka,abEk6rô,abEk6rE,abEk6re,abEk6ra,abEk6rô,...,abEke,abEke,abEke,abEke,abEkjô,abEk,abEkje,abEk,abEk,abEk
7,aberrer,abErE,abEra,abErEr,abEra,abEr6rô,abEr6rE,abEr6re,abEr6ra,abEr6rô,...,NaN,NaN,NaN,abEre,abErjô,abEr,abErje,abEr,abEr,abEr
8,abhorrer,abOrE,abOra,abOrEr,abOra,abOr6rô,abOr6rE,abOr6re,abOr6ra,abOr6rô,...,abOre,abOre,abOre,abOre,abOrjô,abOr,abOrje,abOr,abOr,abOr
9,abjurer,abZyrE,abZyra,abZyrEr,abZyra,abZyr6rô,abZyr6rE,abZyr6re,abZyr6ra,abZyr6rô,...,abZyre,abZyre,abZyre,abZyre,abZyrjô,abZyr,abZyrje,abZyr,abZyr,abZyr


In [82]:
paradigmesGOLD.loc[paradigmesGOLD["lexeme"]=="boire"]["inf"]

627    bwar
Name: inf, dtype: object

##Faire un tableau XML pour la sortie des cliques

class tableauParadigme:
    '''
    Gestion des tableaux de flexion
    '''

    def __init__(self):
        self.paradigme={}
        self.lexeme=""

    def ajouterElement(self,element):
        (lexeme,forme,taminfo)=element.split("-")
        if self.lexeme=="":
            self.lexeme=lexeme
        if self.lexeme==lexeme:
            if len(taminfo)==4:
                (tense,mode,pers,num)=taminfo
            elif taminfo=="inf":
                (tense,mode,pers,num)=("",taminfo,"","")
                
            elif taminfo=="pP":
                (tense,mode,pers,num)=("p","part","","")
            else:
                print("erreur case",element,file=logfile)
            if not mode in self.paradigme:
                self.paradigme[mode]={}
            if not tense in self.paradigme[mode]:
                self.paradigme[mode][tense]={}
            if not pers+num in self.paradigme[mode][tense]:
                self.paradigme[mode][tense][pers+num]=[]
            self.paradigme[mode][tense][pers+num].append(forme)
            
      

seuilClique=25

for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
    if len(clique)>seuilClique:
        print (n, len(clique))
        tabParadigme=tableauParadigme()
        for element in clique:
            tabParadigme.ajouterElement(element)
        for mode in ["i","s","c","I"]:
            for tense in tabParadigme.paradigme[mode]:
                if mode=="I":
                    for pers in ["2S", "1P", "2P"]:
                        print ("-".join(tabParadigme.paradigme[mode][tense][pers]), end=", ")
                else:
                    for pers in [str(i+1)+"S" for i in range(3)]+[str(i+1)+"P" for i in range(3)]:
                        print ("-".join(tabParadigme.paradigme[mode][tense][pers]), end=", ")
                print()